In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
cd /home/dfried/projects/ImageCaptioning.pytorch

/home/dfried/projects/ImageCaptioning.pytorch


In [3]:
def wrap_tag(tag, inner):
    return f'<{tag}>{inner}</{tag}>'

def image_html(image_path, width=300, border=False):
    if border:
        style = ' style="border: 5px solid #0FF" '
    else:
        style = ''
    return f'<img width={width} src="{image_path}" {style}></img>'

def captions_html(captions):
    #return wrap_tag('p', '<br>'.join(' '.join(cap) for cap in captions))
    return wrap_tag('ol', ''.join(wrap_tag('li', cap) for cap in captions))

def images_html(image_paths, width=300, num_per_row=5, target=None, captions=None):
    rows = []
    for ix in range(0, len(image_paths), num_per_row):
        items = [wrap_tag('td', image_html(image_paths[image_ix], width=width, border=image_ix == target)) 
                 for image_ix in range(ix, ix+num_per_row) if image_ix < len(image_paths)]
        rows.append(wrap_tag('tr', ''.join(items)))
        if captions is not None:
            cap_html = [
                wrap_tag('td', captions_html(captions[image_ix]))
                for image_ix in range(ix, ix+num_per_row)
                if image_ix < len(image_paths)
            ]
            rows.append(wrap_tag('tr', ''.join(cap_html)))
    return wrap_tag('table', ''.join(rows))

def display_images(image_paths, width=300, num_per_row=5, target=None, captions=None):
    display(HTML(images_html(image_paths, width=width, num_per_row=num_per_row, target=target, captions=captions)))

In [6]:
def display_neighbors(features, k=5, num_per_row=5):
    neighbor_batch = loader.indices['train'].get_neighbor_batch(loader, features.flatten(), k)
#     neighbor_batch = get_neighbor_batch(features.flatten(), k)
    paths_k = [d['file_path'] for d in neighbor_batch['infos']]
    captions_k = [[' '.join(c) for c in cs] for cs in get_captions_from_batch(neighbor_batch)]
    display_images(paths_k, captions=captions_k, num_per_row=num_per_row)

In [7]:
import torch

In [9]:
def results_name(beam_size, distractors, alpha):
    return f'eval_results/pred_verbose_pragmatic_val_cand-contrastive_beam_search-{beam_size}_distr-{distractors}_alpha-{alpha}_val.pth'

In [14]:
def pretty_name(beam_size, distractors, alpha):
#     return f"bs-{beam_size}_distr-{distractors}_alpha-{alpha}"
    return f"bs-{beam_size}_d-{distractors}_α-{alpha}"

In [42]:
# beam_size, distractors, alpha
systems = [
    (5, 10, 0.0),
    (5, 10, 0.1),
    (5, 10, 0.5),
    (5, 10, 1.0),
    (5, 10, 5.0),
]

In [16]:
results = {
    pretty_name(*t): torch.load(results_name(*t))
    for t in systems
}

In [17]:
results.keys()

dict_keys(['bs-5_d-10_α-0.0', 'bs-5_d-10_α-0.1', 'bs-5_d-10_α-0.5', 'bs-5_d-10_α-1.0', 'bs-5_d-10_α-5.0'])

In [18]:
next(iter(results.values()))[0]

{'image_id': 184613,
 'caption': 'a group of people in a field with animals',
 'perplexity': 0.9350448846817017,
 'entropy': 2.3347628116607666,
 'context_paths': ['val2014/COCO_val2014_000000184613.jpg',
  'val2014/COCO_val2014_000000250804.jpg',
  'train2014/COCO_train2014_000000433662.jpg',
  'train2014/COCO_train2014_000000020966.jpg',
  'train2014/COCO_train2014_000000063043.jpg',
  'train2014/COCO_train2014_000000077693.jpg',
  'train2014/COCO_train2014_000000378214.jpg',
  'train2014/COCO_train2014_000000407590.jpg',
  'val2014/COCO_val2014_000000031471.jpg',
  'train2014/COCO_train2014_000000007953.jpg',
  'train2014/COCO_train2014_000000086329.jpg'],
 'context_ids': [184613,
  250804,
  433662,
  20966,
  63043,
  77693,
  378214,
  407590,
  31471,
  7953,
  86329]}

In [19]:
def all_equal(xs):
    xs = list(xs)
    return all(xs[0] == x for x in xs)

In [55]:
def display_index(index, results, max_distractors=None):
    records = {k: r[index] for k, r in results.items()}
    assert all_equal(r['image_id'] for r in records.values())
#     assert all_equal(len(r['context_paths']) for r in records.values())
    paths = max((d['context_paths'] for d in records.values()), key=len)
#     paths = next(iter(records.values()))['context_paths']
    formatted_captions = [
        "{} | {}".format(r['caption'], name)
        for name, r in records.items()
    ]
    display_images([paths[0]], captions=[formatted_captions], width=400)
    distractors = paths[1:]
    if max_distractors is not None:
        distractors = distractors[:max_distractors]
    display_images(distractors, num_per_row=3)

In [26]:
display_index(0, results)

""
a group of people in a field with animals | bs-5_d-10_α-0.0a group of people in a field with animals | bs-5_d-10_α-0.1a person with a hat and an umbrella | bs-5_d-10_α-0.5a person with a umbrella on a field | bs-5_d-10_α-1.0african animals in a field with people in the background | bs-5_d-10_α-5.0


In [27]:
display_index(1, results)

""
a kitchen with a white stove top oven | bs-5_d-10_α-0.0a kitchen with a white stove top oven | bs-5_d-10_α-0.1a small kitchen with white appliances and white appliances | bs-5_d-10_α-0.5a small kitchen with white appliances and white appliances | bs-5_d-10_α-1.0a small kitchen with white appliances and white appliances | bs-5_d-10_α-5.0


In [28]:
display_index(2, results)

""
a woman holding a cat in her arms | bs-5_d-10_α-0.0a woman holding a cat in her arms | bs-5_d-10_α-0.1a woman holding a cat on a leash | bs-5_d-10_α-0.5a woman holding a cat on a leash | bs-5_d-10_α-1.0toddler playing with a cat on a leash | bs-5_d-10_α-5.0


In [29]:
display_index(3, results)

""
a white toilet sitting in a bathroom next to a wall | bs-5_d-10_α-0.0a white toilet sitting in a bathroom next to a wall | bs-5_d-10_α-0.1a white toilet sitting in a bathroom next to a wall | bs-5_d-10_α-0.5a white toilet in a bathroom next to a wall | bs-5_d-10_α-1.0graffiti on the wall of a small bathroom | bs-5_d-10_α-5.0


In [30]:
display_index(4, results)

""
a kitchen with a sink and a sink | bs-5_d-10_α-0.0a kitchen with a sink and a sink | bs-5_d-10_α-0.1a kitchen with a sink and a sink | bs-5_d-10_α-0.5a sink and a sink in a room | bs-5_d-10_α-1.0fancy kitchen with a stainless steel sink in it | bs-5_d-10_α-5.0


In [31]:
display_index(5, results)

""
a woman riding a bike down a street | bs-5_d-10_α-0.0a woman riding a bike down a street | bs-5_d-10_α-0.1a woman riding a bike down a street | bs-5_d-10_α-0.5a woman riding a bike down a street | bs-5_d-10_α-1.0a woman riding a bike down a street | bs-5_d-10_α-5.0


In [32]:
display_index(6, results)

""
a bath room with a bath tub and a sink | bs-5_d-10_α-0.0a bath room with a bath tub and a sink | bs-5_d-10_α-0.1a bath room with a bath tub and a sink | bs-5_d-10_α-0.5a bath room with a bath tub and a sink | bs-5_d-10_α-1.0brightly colored bathroom with a sink and shower | bs-5_d-10_α-5.0


In [33]:
display_index(7, results)

""
a row of parking meters on a city street | bs-5_d-10_α-0.0a row of parking meters on a city street | bs-5_d-10_α-0.1a row of parking meters on a city street | bs-5_d-10_α-0.5a row of parking meters on a city street | bs-5_d-10_α-1.0row of parking meters lined up on a street | bs-5_d-10_α-5.0


In [34]:
display_index(8, results)

""
a bathroom with two sinks and a bathtub | bs-5_d-10_α-0.0a bathroom with two sinks and a bathtub | bs-5_d-10_α-0.1a bathroom with two sinks and a bathtub | bs-5_d-10_α-0.5a bathroom with two sinks and a bathtub | bs-5_d-10_α-1.0fancy bathroom with two sinks and a bathtub | bs-5_d-10_α-5.0


In [35]:
display_index(9, results)

""
a yellow fire hydrant sitting on top of a dirt field | bs-5_d-10_α-0.0a yellow fire hydrant sitting on top of a pile of dirt | bs-5_d-10_α-0.1a yellow fire hydrant sitting on top of a pile of dirt | bs-5_d-10_α-0.5a bunch of bananas that are on the ground | bs-5_d-10_α-1.0bananas are on the ground next to a fire hydrant | bs-5_d-10_α-5.0


In [36]:
display_index(10, results)

""
a blue truck is parked in the grass | bs-5_d-10_α-0.0a blue truck is parked in the grass | bs-5_d-10_α-0.1a blue truck parked on top of a green field | bs-5_d-10_α-0.5a blue bus is parked in the grass | bs-5_d-10_α-1.0people are standing in front of a bus | bs-5_d-10_α-5.0


In [37]:
display_index(11, results)

""
a black cat drinking water from a toilet | bs-5_d-10_α-0.0a black cat drinking water out of a toilet | bs-5_d-10_α-0.1a black cat sitting on top of a toilet seat | bs-5_d-10_α-0.5a black cat sitting on top of a toilet seat | bs-5_d-10_α-1.0black cat sitting on top of a white toilet seat | bs-5_d-10_α-5.0


In [38]:
display_index(12, results)

""
a tall clock tower with a sky background | bs-5_d-10_α-0.0a tall clock tower with a sky background | bs-5_d-10_α-0.1a tall clock tower with a sky background | bs-5_d-10_α-0.5a tower with a clock on the top of it | bs-5_d-10_α-1.0birds are on the roof of a building | bs-5_d-10_α-5.0


In [39]:
display_index(13, results)

""
a woman riding on the back of a motorcycle down a street | bs-5_d-10_α-0.0a woman riding on the back of a motorcycle down a street | bs-5_d-10_α-0.1a woman riding on the back of a motorcycle down a street | bs-5_d-10_α-0.5a woman riding on the back of a motorcycle down a street | bs-5_d-10_α-1.0woman woman riding on the back of a motorcycle down a street | bs-5_d-10_α-5.0


In [40]:
display_index(14, results)

""
a woman sitting at a desk with a laptop | bs-5_d-10_α-0.0a woman sitting at a desk with a laptop | bs-5_d-10_α-0.1a woman sitting at a desk with a laptop | bs-5_d-10_α-0.5a woman sitting at a desk with a laptop | bs-5_d-10_α-1.0woman sitting at a desk in front of a computer | bs-5_d-10_α-5.0


In [41]:
display_index(15, results)

""
a bunch of bananas that are in a basket | bs-5_d-10_α-0.0a bunch of bananas that are in a basket | bs-5_d-10_α-0.1a bunch of bananas that are on a table | bs-5_d-10_α-0.5a bunch of bananas that are on a table | bs-5_d-10_α-1.0bunches of bananas are on display in a store | bs-5_d-10_α-5.0


In [54]:
results_diff_distractors = {
    pretty_name(*t): torch.load(results_name(*t))
    # beam_size, distractors, alpha
    for t in [
    (5, 1, 0.0),
    (5, 1, 1.0),
    (5, 5, 1.0),
    (5, 10, 1.0),
    (5, 10, 5.0),
    (5, 50, 1.0),
#     (5, 50, 5.0),
]}

In [56]:
display_index(0, results_diff_distractors, max_distractors=5)

""
a group of people in a field with animals | bs-5_d-1_α-0.0a woman standing next to a herd of cattle | bs-5_d-1_α-1.0a woman standing next to a herd of cattle | bs-5_d-5_α-1.0a person with a umbrella on a field | bs-5_d-10_α-1.0african animals in a field with people in the background | bs-5_d-10_α-5.0a young boy standing next to a herd of cattle | bs-5_d-50_α-1.0


In [57]:
display_index(1, results_diff_distractors, max_distractors=5)

""
a kitchen with a white stove top oven | bs-5_d-1_α-0.0a small kitchen with white appliances and white appliances | bs-5_d-1_α-1.0a kitchen with white appliances and white appliances | bs-5_d-5_α-1.0a small kitchen with white appliances and white appliances | bs-5_d-10_α-1.0a small kitchen with white appliances and white appliances | bs-5_d-10_α-5.0a small kitchen with white appliances and white appliances | bs-5_d-50_α-1.0


In [58]:
display_index(2, results_diff_distractors, max_distractors=5)

""
a woman holding a cat in her arms | bs-5_d-1_α-0.0a woman holding a cat in her arms | bs-5_d-1_α-1.0a woman holding a cat on a leash | bs-5_d-5_α-1.0a woman holding a cat on a leash | bs-5_d-10_α-1.0toddler playing with a cat on a leash | bs-5_d-10_α-5.0a woman holding a cat on a leash | bs-5_d-50_α-1.0


In [59]:
display_index(3, results_diff_distractors, max_distractors=5)

""
a white toilet sitting in a bathroom next to a wall | bs-5_d-1_α-0.0a white toilet in a bathroom next to a wall | bs-5_d-1_α-1.0a white toilet sitting in a bathroom next to a wall | bs-5_d-5_α-1.0a white toilet in a bathroom next to a wall | bs-5_d-10_α-1.0graffiti on the wall of a small bathroom | bs-5_d-10_α-5.0a white toilet in a bathroom next to a wall | bs-5_d-50_α-1.0


In [60]:
display_index(4, results_diff_distractors, max_distractors=5)

""
a kitchen with a sink and a sink | bs-5_d-1_α-0.0a bathroom with a sink and a sink | bs-5_d-1_α-1.0a small kitchen with a stainless steel sink | bs-5_d-5_α-1.0a sink and a sink in a room | bs-5_d-10_α-1.0fancy kitchen with a stainless steel sink in it | bs-5_d-10_α-5.0a large sink in a very small bathroom | bs-5_d-50_α-1.0


In [61]:
display_index(5, results_diff_distractors, max_distractors=5)

""
a woman riding a bike down a street | bs-5_d-1_α-0.0a woman riding a bike down a street | bs-5_d-1_α-1.0a woman riding a bike down a street | bs-5_d-5_α-1.0a woman riding a bike down a street | bs-5_d-10_α-1.0a woman riding a bike down a street | bs-5_d-10_α-5.0a woman riding a bike down a street | bs-5_d-50_α-1.0


In [62]:
display_index(6, results_diff_distractors, max_distractors=5)

""
a bath room with a bath tub and a sink | bs-5_d-1_α-0.0a bath room with a bath tub and a sink | bs-5_d-1_α-1.0a bath room with a bath tub and a sink | bs-5_d-5_α-1.0a bath room with a bath tub and a sink | bs-5_d-10_α-1.0brightly colored bathroom with a sink and shower | bs-5_d-10_α-5.0a bathroom with a blue shower curtain in it | bs-5_d-50_α-1.0


In [63]:
display_index(7, results_diff_distractors, max_distractors=5)

""
a row of parking meters on a city street | bs-5_d-1_α-0.0a close up of a parking meter on a street | bs-5_d-1_α-1.0a row of parking meters on a city street | bs-5_d-5_α-1.0a row of parking meters on a city street | bs-5_d-10_α-1.0row of parking meters lined up on a street | bs-5_d-10_α-5.0a row of parking meters on a city street | bs-5_d-50_α-1.0


In [64]:
display_index(8, results_diff_distractors, max_distractors=5)

""
a bathroom with two sinks and a bathtub | bs-5_d-1_α-0.0a bathroom with two sinks and a bathtub | bs-5_d-1_α-1.0a bathroom with two sinks and a bathtub | bs-5_d-5_α-1.0a bathroom with two sinks and a bathtub | bs-5_d-10_α-1.0fancy bathroom with two sinks and a bathtub | bs-5_d-10_α-5.0a bathroom with two sinks and a bathtub | bs-5_d-50_α-1.0


In [65]:
display_index(9, results_diff_distractors, max_distractors=5)

""
a yellow fire hydrant sitting on top of a dirt field | bs-5_d-1_α-0.0a close up of a yellow fire hydrant | bs-5_d-1_α-1.0there is a yellow and blue fire hydrant on the ground | bs-5_d-5_α-1.0a bunch of bananas that are on the ground | bs-5_d-10_α-1.0bananas are on the ground next to a fire hydrant | bs-5_d-10_α-5.0a ripe fire hydrant sitting on top of a pile of dirt | bs-5_d-50_α-1.0


In [66]:
display_index(10, results_diff_distractors, max_distractors=5)

""
a blue truck is parked in the grass | bs-5_d-1_α-0.0a large blue truck parked in a field | bs-5_d-1_α-1.0a blue bus parked on top of a green field | bs-5_d-5_α-1.0a blue bus is parked in the grass | bs-5_d-10_α-1.0people are standing in front of a bus | bs-5_d-10_α-5.0a blue bus parked on top of a green field | bs-5_d-50_α-1.0


In [67]:
display_index(11, results_diff_distractors, max_distractors=5)

""
a black cat drinking water from a toilet | bs-5_d-1_α-0.0a black cat sitting on top of a white toilet seat | bs-5_d-1_α-1.0a black cat sitting on top of a white toilet seat | bs-5_d-5_α-1.0a black cat sitting on top of a toilet seat | bs-5_d-10_α-1.0black cat sitting on top of a white toilet seat | bs-5_d-10_α-5.0a black cat sitting on top of a white toilet seat | bs-5_d-50_α-1.0


In [68]:
display_index(12, results_diff_distractors, max_distractors=5)

""
a tall clock tower with a sky background | bs-5_d-1_α-0.0a tall clock tower with a sky background | bs-5_d-1_α-1.0a building with a clock on the top of it | bs-5_d-5_α-1.0a tower with a clock on the top of it | bs-5_d-10_α-1.0birds are on the roof of a building | bs-5_d-10_α-5.0a building with a clock on the top of it | bs-5_d-50_α-1.0


In [69]:
display_index(13, results_diff_distractors, max_distractors=5)

""
a woman riding on the back of a motorcycle down a street | bs-5_d-1_α-0.0a woman riding on the back of a motorcycle down a street | bs-5_d-1_α-1.0a woman riding on the back of a motorcycle down a street | bs-5_d-5_α-1.0a woman riding on the back of a motorcycle down a street | bs-5_d-10_α-1.0woman woman riding on the back of a motorcycle down a street | bs-5_d-10_α-5.0a woman riding on the back of a motorcycle down a street | bs-5_d-50_α-1.0


In [70]:
display_index(14, results_diff_distractors, max_distractors=5)

""
a woman sitting at a desk with a laptop | bs-5_d-1_α-0.0a woman sitting at a desk with a laptop | bs-5_d-1_α-1.0a woman sitting at a desk with a laptop | bs-5_d-5_α-1.0a woman sitting at a desk with a laptop | bs-5_d-10_α-1.0woman sitting at a desk in front of a computer | bs-5_d-10_α-5.0a woman sitting at a desk with a laptop | bs-5_d-50_α-1.0


In [71]:
display_index(15, results_diff_distractors, max_distractors=5)

""
a bunch of bananas that are in a basket | bs-5_d-1_α-0.0a bunch of bananas that are in a basket | bs-5_d-1_α-1.0a bunch of bananas that are in a basket | bs-5_d-5_α-1.0a bunch of bananas that are on a table | bs-5_d-10_α-1.0bunches of bananas are on display in a store | bs-5_d-10_α-5.0a bunch of bananas wrapped in plastic bags | bs-5_d-50_α-1.0


In [72]:
display_index(16, results_diff_distractors, max_distractors=5)

""
a fighter jet sitting on top of an airport tarmac | bs-5_d-1_α-0.0a fighter jet sitting on top of an airport tarmac | bs-5_d-1_α-1.0a blue and white airplane is on a runway | bs-5_d-5_α-1.0a blue fighter jet is on a runway | bs-5_d-10_α-1.0a blue fighter jet is on a runway | bs-5_d-10_α-5.0a blue and yellow fighter plane on a runway | bs-5_d-50_α-1.0


In [73]:
display_index(17, results_diff_distractors, max_distractors=5)

""
a large jetliner sitting on top of an airport tarmac | bs-5_d-1_α-0.0a large passenger jet sitting on top of an airport tarmac | bs-5_d-1_α-1.0a large jetliner sitting on top of an airport tarmac | bs-5_d-5_α-1.0a group of airplanes that are on a runway | bs-5_d-10_α-1.0two airplanes that are parked in a lot | bs-5_d-10_α-5.0a group of airplanes that are on a runway | bs-5_d-50_α-1.0


In [74]:
display_index(18, results_diff_distractors, max_distractors=5)

""
a car is stopped at a red light | bs-5_d-1_α-0.0a car is stopped at a red light | bs-5_d-1_α-1.0a car is stopped at a red light | bs-5_d-5_α-1.0a car is stopped at a red light | bs-5_d-10_α-1.0a white car stopped at a traffic light | bs-5_d-10_α-5.0a car is stopped at a red light | bs-5_d-50_α-1.0


In [75]:
display_index(19, results_diff_distractors, max_distractors=5)

""
a traffic light sitting on the side of a road | bs-5_d-1_α-0.0a green traffic light sitting on the side of a road | bs-5_d-1_α-1.0a traffic light with a green light on it | bs-5_d-5_α-1.0a traffic light sitting next to a traffic light | bs-5_d-10_α-1.0two green traffic lights are on a pole | bs-5_d-10_α-5.0a green traffic light sitting next to a traffic light | bs-5_d-50_α-1.0


In [76]:
display_index(20, results_diff_distractors, max_distractors=5)

""
a red and white airplane flying in the sky | bs-5_d-1_α-0.0a red and white airplane flying in the sky | bs-5_d-1_α-1.0a red and white airplane flying in the sky | bs-5_d-5_α-1.0a red and white airplane flying in the sky | bs-5_d-10_α-1.0red and white airplane flying in the sky | bs-5_d-10_α-5.0a red and white airplane flying in the sky | bs-5_d-50_α-1.0


In [77]:
display_index(21, results_diff_distractors, max_distractors=5)

""
a city street filled with lots of tall buildings | bs-5_d-1_α-0.0a city street filled with lots of tall buildings | bs-5_d-1_α-1.0a city street filled with lots of tall buildings | bs-5_d-5_α-1.0a city street filled with lots of tall buildings | bs-5_d-10_α-1.0people crossing the street in a large city | bs-5_d-10_α-5.0a city street with tall buildings in the background | bs-5_d-50_α-1.0


In [78]:
display_index(22, results_diff_distractors, max_distractors=5)

""
a blue and white bus parked in a parking lot | bs-5_d-1_α-0.0a truck with a mural on the side of it | bs-5_d-1_α-1.0a truck with a mural on the side of it | bs-5_d-5_α-1.0a blue bus with a advertisement on the side of it | bs-5_d-10_α-1.0a van with a mural on the side of it | bs-5_d-10_α-5.0a blue truck with a advertisement on the side of it | bs-5_d-50_α-1.0


In [79]:
display_index(23, results_diff_distractors, max_distractors=5)

""
a couple of giraffe standing next to each other | bs-5_d-1_α-0.0two giraffes standing next to each other in a field | bs-5_d-1_α-1.0two giraffes are standing in a grassy field | bs-5_d-5_α-1.0two giraffes are standing in a grassy field | bs-5_d-10_α-1.0two baby giraffes standing next to each other | bs-5_d-10_α-5.0two giraffes are standing in a grassy field | bs-5_d-50_α-1.0


In [80]:
display_index(24, results_diff_distractors, max_distractors=5)

""
a giraffe is laying down in the dirt | bs-5_d-1_α-0.0a giraffe is laying down in the dirt | bs-5_d-1_α-1.0a giraffe laying on the ground next to a building | bs-5_d-5_α-1.0a giraffe laying on the ground next to a building | bs-5_d-10_α-1.0a baby giraffe laying down on the ground | bs-5_d-10_α-5.0a giraffe laying down on a dirt ground | bs-5_d-50_α-1.0


In [81]:
display_index(100, results_diff_distractors, max_distractors=5)

""
a young boy laying on top of a bed | bs-5_d-1_α-0.0a young boy laying on top of a bed | bs-5_d-1_α-1.0a young boy laying on top of a bed | bs-5_d-5_α-1.0a young boy laying on top of a bed | bs-5_d-10_α-1.0young boy laying on a bed with a cell phone | bs-5_d-10_α-5.0a little boy that is laying down on a bed | bs-5_d-50_α-1.0


In [82]:
display_index(101, results_diff_distractors, max_distractors=5)

""
a living room filled with furniture and a flat screen tv | bs-5_d-1_α-0.0a living room filled with furniture and a large window | bs-5_d-1_α-1.0a living room filled with furniture and a fire place | bs-5_d-5_α-1.0a living room filled with furniture and a sliding glass door | bs-5_d-10_α-1.0an empty living room filled with wooden furniture | bs-5_d-10_α-5.0a living room filled with furniture and a large window | bs-5_d-50_α-1.0


In [83]:
display_index(102, results_diff_distractors, max_distractors=5)

""
a bunch of kites flying in the sky | bs-5_d-1_α-0.0a bunch of kites flying in the sky | bs-5_d-1_α-1.0a bunch of kites flying in the sky | bs-5_d-5_α-1.0a bunch of kites flying in the sky | bs-5_d-10_α-1.0a bunch of kites flying in the sky | bs-5_d-10_α-5.0a bunch of kites flying in the sky | bs-5_d-50_α-1.0


In [84]:
display_index(103, results_diff_distractors, max_distractors=5)

""
a man flying a kite in the snow | bs-5_d-1_α-0.0a man flying a kite in the snow | bs-5_d-1_α-1.0a man flying a kite in the snow | bs-5_d-5_α-1.0a man flying a kite in the snow | bs-5_d-10_α-1.0man flying a kite on a snow covered slope | bs-5_d-10_α-5.0a man flying a kite on top of a snow covered slope | bs-5_d-50_α-1.0


In [85]:
display_index(104, results_diff_distractors, max_distractors=5)

""
a group of laptops sitting on top of a wooden table | bs-5_d-1_α-0.0a group of laptops sitting on top of a wooden table | bs-5_d-1_α-1.0a room filled with lots of laptop computers | bs-5_d-5_α-1.0a table with three laptops on it | bs-5_d-10_α-1.0four laptops are sitting on a small table | bs-5_d-10_α-5.0a table with several laptops on it | bs-5_d-50_α-1.0


In [86]:
display_index(105, results_diff_distractors, max_distractors=5)

""
a large clock tower towering over a city | bs-5_d-1_α-0.0a large clock tower towering over a city | bs-5_d-1_α-1.0a large clock tower towering over a city | bs-5_d-5_α-1.0a large clock tower towering over a city | bs-5_d-10_α-1.0a clock tower over a body of water | bs-5_d-10_α-5.0a large clock tower towering over a city | bs-5_d-50_α-1.0


In [87]:
display_index(106, results_diff_distractors, max_distractors=5)

""
a baseball player swinging a bat at a ball | bs-5_d-1_α-0.0a baseball player swinging a bat at a ball | bs-5_d-1_α-1.0a baseball player swinging a bat at a ball | bs-5_d-5_α-1.0a baseball player swinging a bat at a ball | bs-5_d-10_α-1.0a boy swinging a bat at a baseball | bs-5_d-10_α-5.0a baseball player swinging a bat at a ball | bs-5_d-50_α-1.0
